In [1]:
import numpy as np
import urllib
import os

# Download the dataset

In [ ]:
# Get the webpage
url = 'https://landsat.usgs.gov/landsat-8-cloud-cover-assessment-validation-data'
response = urllib.request.urlopen(url)

# Find the data urls in the source
data_urls = []
for line in response:
    if 'https://landsat.usgs.gov/cloud-validation/cca' in str(line) and 'tar' in str(line):        
        data_urls.append(str(line)[47:117])

print("Products found:", np.size(data_urls))
print("Example url:   ", data_urls[0])        

Products found: 96
Example url:    https://landsat.usgs.gov/cloud-validation/cca_l8/LC80420082013220LGN00.tar.gz


In [ ]:
# Download all the products
backgrounds = ['Barren',
               'Forest',
               'GrassCrops',
               'Shrubland',
               'SnowIce',
               'Urban',
               'Water',
               'Wetlands']
index = -1
for i in range(0, np.size(data_urls)):
    if i%12 == 0:  # There are 12 scenes in every background category       
        index += 1     
    !wget {data_urls[i]+'.tar.gz'} -P {'./tar/' + backgrounds[index] + '/'}
    

In [10]:
# Unzip all files into their respective folders
folders = sorted(os.listdir("tar"))

for folder in folders: 
    !mkdir {"./unzipped/" + folder}
    files = sorted(os.listdir("tar/" + folder))
    files = [f for f in files if '.tar.gz' in f] 
    for file in files:
        try: 
            print("----")
            print("Unzipping product:", file)
            !tar -xvzf {"./tar/" + folder + '/' + file} -C {'./unzipped/' + folder + '/'}
        except:
            print("----------------------------------------------------")
            print("UNZIP ERROR WITH PRODUCT:", file)
            pass

----
Unzipping product: LC80420082013220LGN00.tar.gz
BC/LC80420082013220LGN00/LC80420082013220LGN00_B10.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B11.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B1.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B2.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B3.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B4.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B5.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B6.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B7.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B8.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_B9.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_BQA.TIF
BC/LC80420082013220LGN00/LC80420082013220LGN00_fixedmask.hdr
BC/LC80420082013220LGN00/LC80420082013220LGN00_fixedmask.img
BC/LC80420082013220LGN00/LC80420082013220LGN00_MTL.txt
----
Unzipping product: LC80500092014231LGN00.tar.gz
BC/LC80500092014231LGN00/LC80500092014231LGN00_B10.TIF
BC/LC805000

In [15]:
# Convert the .img masks to .TIF masks
folders = sorted(os.listdir("./unzipped/"))
folders = [f for f in folders if '.' not in f]  # Filter out .gitignore
for folder in folders:
    products = sorted(os.listdir("./unzipped/" + folder + "/BC/"))
    for product in products:
        img_path  = "./unzipped/" + folder + "/BC/" + product + "/" + product + "_fixedmask.img"
        gtiff_path = "./unzipped/" + folder + "/BC/" + product + "/" + product + "_fixedmask.TIF"
        !gdal_translate -of GTiff {img_path + " " + gtiff_path}

Input file size is 8911, 8941
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 8671, 8721
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 9101, 9071
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7611, 7751
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7871, 7991
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7781, 7901
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 8931, 8961
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7551, 7721
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7551, 7741
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7571, 7391
0...10...20...30...40...50...60...70...80...90...100 - done.
Input file size is 7511, 7321
0...10...20...30...40...50...60...70...80...90...100 - done.

In [12]:
# Move to raw data folder
project_path = "/home/jhj/phd/GitProjects/SentinelSemanticSegmentation/"
!cp -R ./unzipped/* {project_path + 'data/raw/'}

In [ ]:
# Download SPARCS dataset
!wget https://landsat.usgs.gov/cloud-validation/sparcs/l8cloudmasks.zip -P ./zip/